# Y21ss Ingest

In [ ]:
# Initialize
import pandas as pd
import numpy as np
import math
import time
import json
import h5py

# Define function to serialize output file names
def serial(pad): # generate a zero padded serial number string
    sn_file = 'SN.txt'
    f = open(sn_file, "r")
    n = int(f.read())    
    f = open(sn_file, "w")
    f.write(str(n+1))
    f.close()
    return str(n).zfill(pad)

# Make some choices
geneset = 'nMod' # 'nMod' (785 neuromodulatory genes), 'NP' (Canonical NPP and NP-GPCR genes) or 'PC'
#regions_common = ['VIS', 'AUD', 'SSp', 'MOp', 'AI', 'ACA', 'RSP', 'TEa-PERI-ECT', 'HIP']
# Decomment one
#regions = regions_common # iterate through all common regions
#regions = ['MOp', 'HIP', 'SSp', 'ACA', 'VIS'] # iterate through most populous common regions
#regions = ['MOp'] # single most populous of common regions
regions = ['Union'] # aggregate of all regions

# Check for possibly impossible memory requirement
if geneset == 'PC' and regions == ['Union']: print('*** BEWARE OF EXCESS MEMORY USAGE! ***')

# Choose source files
read_path = '/Users/stephens/Primary_scRNAseq_Data/Y21ss/expression_matrix.csv'
metadata_path = '/Users/stephens/Primary_scRNAseq_Data/Y21ss/metadata.csv'
print(' Expression data read file:', read_path)
print('        Metadata read file:', metadata_path)
gene_sifter_path = 'Resources/All_mouse_'+geneset+'_genes.csv'
print('          Gene sifter file:', gene_sifter_path)
pc_gene_path = 'Resources/All_mouse_PC_genes.csv'
print('  Protein-coding gene file:', pc_gene_path)


In [ ]:
# Create annotation dataframe

# Create annotation import dictionary and sifter
anno_import_dict = {'class_label': 'class',
                    'class_order': 'class_order',
                    'class_color': 'class_color',
                    'neighborhood_label': 'neighborhood',
                    'neighborhood_id': 'neighborhood_order',
                    'neighborhood_color': 'neighborhood_color',
                    'subclass_label': 'subclass',
                    'subclass_order': 'subclass_order',
                    'subclass_color': 'subclass_color',
                    'cluster_label': 'cluster',
                    'cluster_order': 'cluster_order',
                    'cluster_color': 'cluster_color',
                    'region_label': 'region',
                    'region_id': 'region_order',
                    'region_color': 'region_color',
                    'sample_name': 'sample_name'}
anno_sifter = list(anno_import_dict.keys())

# Import and rename annotation columns
annos = pd.read_csv(metadata_path, index_col='sample_name', usecols=anno_sifter, low_memory=False)
annos = annos.rename(columns=anno_import_dict).reset_index().iloc[:, ::-1]
annos_list = list(anno_import_dict.values())
annos = annos[annos_list]
annos.set_index('sample_name', inplace=True)


In [ ]:
# Initialize annotation support dict and metadata items list

support_dict = {}
major_annos = ['class','neighborhood','subclass','cluster','region']
df = annos.set_index(major_annos)
for anno in major_annos:
    items = df.groupby(anno).count()[anno+'_order']
    names = list(items.index)
    tallies = [int(i) for i in list(items.values)]
    support_dict.update({anno+'_names': names})
    support_dict.update({anno+'_tallies': tallies})    
write_annots = ['type','region','class','neighborhood','subclass','cluster',
                'region_color','class_color','neighborhood_color','subclass_color','cluster_color',
                'genes_mapped','counts_sum','ordinal']
support_dict.update({'annotation_labels': write_annots})
drop_annots = ['class_order','neighborhood_order','subclass_order','cluster_order','region_order','sample_name']
type_dict = {'Glutamatergic': '01 Neuronal', 'GABAergic': '01 Neuronal', 'Non-Neuronal': '02 Non-Neuronal'}


In [ ]:
# Transform metadata columns

df = annos.copy()
# create new composite columns
df.insert(0, 'ordinal', 0)
df.insert(1, 'type', [type_dict[s] for s in list(df['class'])])
df['region_order'] = [f"{x:03d}" for x in list( df['region_order'])]
df['region'] = df['region_order']+' '+df['region']
df['class_order'] = [f"{x:02d}" for x in list( df['class_order'])]
df['class'] = df['class_order']+' '+df['class']
df['neighborhood_order'] = [f"{x:02d}" for x in list( df['neighborhood_order'])]
df['neighborhood'] = df['neighborhood_order']+' '+df['neighborhood']
df['subclass_order'] = [f"{x:02d}" for x in list( df['subclass_order'])]
df['subclass'] = df['subclass_order']+' '+df['subclass']
df['cluster_order'] = [f"{x:03d}" for x in list( df['cluster_order'])]
df['cluster'] = [s.split('_')[1] for s in list( df['cluster'])]
df['cluster'] = df['cluster_order']+' '+df['cluster']
# relabel RSPv rows as RSP 
df['region'] = ['020 RSP' if x == '029 RSPv' else x for x in df['region']]
annos = df.copy()


In [ ]:
# Read gene lists

# Read list of protein-coding genes
support_dict.update({'missing_pcs': 'None'})
pc_genes = list(pd.read_csv(pc_gene_path)['Symbol'])
#pc_genes = [x for x in pc_genes if x not in missing_pcs]

# Read list of selected genes
support_dict.update({'missing_nMods': 'None'})
gene_list = list(pd.read_csv(gene_sifter_path)['Symbol'])
#gene_list = [x for x in gene_list if x not in missing_nMods]

# Test for genes missing in source matrix (none missing in this source!)
df = pd.read_csv(read_path, index_col='sample_name', nrows=2)
raw_genes = list(df.columns)
df = df[pc_genes]

print(len(pc_genes), len(gene_list))


In [ ]:
# Read counts and sift to memory

sn = serial(5)
chunk_size = 5000

start = time.time()

print('\n',regions,'\n')
for region in regions:
    print(region)
    # Set output file paths
    json_path = 'Products/Y21ss_'+geneset+'_'+region+'_support_'+sn+'.json'
    print('              Support path:', json_path)
    csv_path = 'Products/Y21ss_'+geneset+'_'+region+'_counts_'+sn+'.csv'
    print('Expression data write path:', csv_path)
    counts = pd.DataFrame()
    for chunk in pd.read_csv(read_path, index_col='sample_name',
                             usecols=['sample_name']+pc_genes, chunksize=chunk_size):
        sums = chunk.sum(axis=1)
        genes = chunk[chunk > 0].count(axis=1)
        if region != 'Union': chunk = chunk[chunk['region'] == region]
        chunk = annos.merge(chunk, on='sample_name')        
        chunk = chunk[gene_list]
        if len(chunk) > 0:
            chunk.insert(loc=0, column='counts_sum', value=sums)
            chunk.insert(loc=0, column='genes_mapped', value=genes)
            counts = pd.concat([counts, chunk])
        print("\r%5.1f seconds    " % (time.time()-start), counts.shape)
    print('\n')
    
    # Merge annotations with counts and update annotation columns
    df = annos.merge(counts, on='sample_name')
    df.reset_index(inplace=True)
    df.drop(columns=drop_annots, inplace=True)
    df = df[write_annots+gene_list]

    df.set_index(write_annots, inplace=True)
    df.sort_index(level=['type','region','class','neighborhood','subclass','cluster'], inplace=True)
    df.reset_index('ordinal', inplace=True)
    df['ordinal'] = range(len(df))
    df.set_index('ordinal', append=True, inplace=True)
    ann_data = df.copy()

    # Write out support dictionary
    with open(json_path, 'w') as fp:
        json.dump(support_dict, fp)

    # Write out trimmed and annotated expression data
    ann_data.to_csv(csv_path+'.zip')

print("\nFini! %5.1f seconds    " % (time.time()-start))

In [ ]:
# Readback expression data
start = time.time()
df = pd.read_csv(csv_path+'.zip')
end = time.time()
print("Fini! %5.1f seconds    " % (end-start))
# Sanity check
print(df[['Cck','Npy','Vip','Cckbr','Kng1','Bdkrb1','Bdkrb2']].sum())


In [ ]:
# Sanity check
counts[['Cck','Npy','Vip','Cckbr','Kng1','Bdkrb1','Bdkrb2']].sum()


In [ ]:
# Readback support file
with open(json_path, 'r') as fp:
    temp = json.load(fp)#['annotation_labels']
temp
